### Importing Libraries

In [ ]:
import os
import json
import boto3
import re

import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

### Creating config

In [ ]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name
role = get_execution_role()

### Setting Path to S3

In [ ]:
S3_BUCKET = "sagemaker-taxi-fare"

TRAIN_FOLDER  = r"train/"
TEST_FOLDER  = r"test/"
MODEL_FOLDER  = r"model/"

TRAIN_DATA_URI = r"s3://" + S3_BUCKET + r"/" + TRAIN_FOLDER
TEST_DATA_URI = r"s3://" + S3_BUCKET + r"/" + TEST_FOLDER

### Prepare files required to build the containers

In [ ]:
REPOSITORY = "taxi-fare"

In [ ]:
!mkdir build
!cp Dockerfile build/
!cp requirements.txt build/
!mkdir build/src
!cp src/* build/src/

In [ ]:
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.session.Session().region_name

os.environ["AWS_ACCOUNT_ID"] = account_id
os.environ["AWS_REGION"] = region
os.environ["REPO_NAME"] = REPOSITORY

In [ ]:
%%sh

cd build
docker build -t $REPO_NAME .
aws ecr get-login-password --region $AWS_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com
aws ecr create-repository --repository-name $REPO_NAME
docker tag $REPO_NAME:latest $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com/$REPO_NAME:latest
docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com/$REPO_NAME:latest
cd ..
rm -r build/

### Training Estimator

In [ ]:
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY}:latest"

In [ ]:
estimator = Estimator(
    image_uri=image_uri,
    role=get_execution_role(),
    base_job_name="taxi-fare-v1",
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters = {'epochs': 10, 'batch-size': 10, 'learning-rate': 0.01, 'verbose': 1}
)

In [ ]:
estimator.fit({
    "train": TRAIN_DATA_URI,
    "test": TEST_DATA_URI
})

### Deploying Estimator

In [ ]:
ENDPOINT_NAME = "taxi-fare-nyc"
runtime = boto3.client('runtime.sagemaker')

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=ENDPOINT_NAME
)

In [ ]:
input_data = {
    None
}

In [ ]:
response  = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(input_data)
)

In [ ]:
result = json.loads(response['Body'].read().decode())
result

Cleaning up endpoint once we are done with it

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint)